In [ ]:
!pip install openai
!pip install dotenv

In [8]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['API2D_API_KEY']
openai.api_base = 'https://openai.api2d.net/v1'

In [4]:
# 用户的商品评价
customer_review = """
MacBook Pro特别棒！特别喜欢！m2处理器性能超强，就是价钱有点小贵！电池续航逆天！不发热！还带有黑科技触控栏！这是我送给奶奶的礼物，但是花了好三天才到。
"""

# Prompt 编写
review_template = """\
从以下的文本提取信息:

gift: 是不是为某个人购买的礼物？如果是的回答True，如果不是回答False
delivery_days: 花了几天收到了礼物？输出一个数字，如果没有这个信息，输出-1
price_value: 获取这个物品的价格或者价值，如果有多个，用逗号分隔组成一个python数组

用以下的键值来格式化信息并输出一个JSON:
gift
delivery_days
price_value

文本: {text}
"""


In [28]:
def get_next_date():
    return "我准备端午节的时候上新"
    
functions = [
    {
        "name": "getNextDate",
        "description": "获取上新时间",
        "parameters": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string",
                    "description": "童装款式",
                }
            },
            "required": [],
        },
    }
]
available_functions = {
    "getNextDate": get_next_date,
} 


In [32]:
import json

messages = [
        {"role": "system", "content": "你现在是一个童装网店的客服"},
        {"role": "user", "content": "什么时候上新"}
    ]
rsp = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  functions=functions,
  function_call="auto",
  messages=messages
)
response_message=rsp["choices"][0]["message"]
function_name = response_message["function_call"]["name"]
fuction_to_call = available_functions[function_name]
function_args = json.loads(response_message["function_call"]["arguments"])
# 用这种方式可以调起任意python函数，不用像官网那样还要指定参数名  
function_response = fuction_to_call(**function_args)
# 获取到函数调用结果后，需要将结果拼接到对话记录里，并再次调用ChatGPT
messages.append(response_message) 
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
) 
# 二次调用的返回结果里就是我们预期的结果了 
response2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
rtn_message = response2["choices"][0]["message"]
print(rtn_message.get('content'))

我们计划在端午节期间推出一批新款童装。请您留意我们网店的最新消息和更新，以获取详细的上新时间和产品。如果您有其他问题或需要帮助，随时都可以向我咨询。谢谢！
